In [1]:
from path import Path
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import cross_val_score

In [2]:
data = Path('Resources/kc_house_data.csv')
df = pd.read_csv(data).set_index("id" )
df.head(20)

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,,,,,,,,,,,,,,,,,,,,
7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
7237550310,20140512T000000,1225000.0,4,4.50,5420,101930,1.0,0,0,3,11,3890,1530,2001,0,98053,47.6561,-122.005,4760,101930
1321400060,20140627T000000,257500.0,3,2.25,1715,6819,2.0,0,0,3,7,1715,0,1995,0,98003,47.3097,-122.327,2238,6819
2008000270,20150115T000000,291850.0,3,1.50,1060,9711,1.0,0,0,3,7,1060,0,1963,0,98198,47.4095,-122.315,1650,9711
2414600126,20150415T000000,229500.0,3,1.00,1780,7470,1.0,0,0,3,7,1050,730,1960,0,98146,47.5123,-122.337,1780,8113


In [3]:
df['date'] = pd.to_datetime(df['date']) #Dropping date column bcz i won't gonna day and month and creating new year column
df['year'] = df['date'].dt.year
df = df.drop("date",axis=1)
df

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,year
id,,,,,,,,,,,,,,,,,,,,
7129300520,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,2014
6414100192,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,2014
5631500400,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062,2015
2487200875,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000,2014
1954400510,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263000018,360000.0,3,2.50,1530,1131,3.0,0,0,3,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509,2014
6600060120,400000.0,4,2.50,2310,5813,2.0,0,0,3,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200,2015
1523300141,402101.0,2,0.75,1020,1350,2.0,0,0,3,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007,2014


In [4]:
# X = df.drop(['date', 'price', 'zipcode', 'lat', 'long'], axis = 1)
X = df.drop(['price'], axis = 1)

In [5]:
y = df["price"]
# X = df.drop(columns="price")

In [6]:
y.head()

id
7129300520    221900.0
6414100192    538000.0
5631500400    180000.0
2487200875    604000.0
1954400510    510000.0
Name: price, dtype: float64

In [7]:
X.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,year
id,,,,,,,,,,,,,,,,,,,
7129300520,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,2014
6414100192,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,2014
5631500400,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062,2015
2487200875,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000,2014
1954400510,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503,2015


In [8]:
import statsmodels.api as sm

In [9]:
mod = sm.OLS(y, X)

In [10]:
res = mod.fit()

In [11]:
print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:                  price   R-squared (uncentered):                   0.905
Model:                            OLS   Adj. R-squared (uncentered):              0.905
Method:                 Least Squares   F-statistic:                          1.147e+04
Date:                Sat, 13 Nov 2021   Prob (F-statistic):                        0.00
Time:                        11:41:05   Log-Likelihood:                     -2.9458e+05
No. Observations:               21613   AIC:                                  5.892e+05
Df Residuals:                   21595   BIC:                                  5.893e+05
Df Model:                          18                                                  
Covariance Type:            nonrobust                                                  
                    coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [13]:
X_train.shape

(16209, 19)

In [14]:
# MAchine lerning model Linear Regression.
from sklearn.linear_model import LinearRegression

linear_reg = LinearRegression().fit(X_train, y_train)

In [32]:
y_pred = linear_reg.predict(X_test)
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test})
results.head(5)

,Prediction,Actual
id,,
2591820310,4.508318e+05,365000.0
7974200820,7.425032e+05,865000.0
7701450110,1.229031e+06,1038000.0
9522300010,1.683019e+06,1490000.0
9510861140,7.253682e+05,711000.0


In [16]:
cross_val_score(linear_reg, X, y, cv=10)

array([0.69691721, 0.69946573, 0.69050271, 0.69301439, 0.68227256,
       0.70736821, 0.71352467, 0.7098618 , 0.69974197, 0.67488055])

In [17]:
# Linear Regression score
linear_reg.score(X, y, sample_weight=None)

0.7010593164544259

In [18]:
# Machinbe learning model Random Tree

In [19]:
from sklearn.tree import DecisionTreeRegressor

In [20]:
regressor = DecisionTreeRegressor(random_state=79)

In [21]:
# Machinbe learning model Random tree
y_pred = regressor.fit(X_train, y_train).predict(X_test)
results_tree = pd.DataFrame({"Prediction": y_pred, "Actual": y_test})
results_tree.head(5)

,Prediction,Actual
id,,
2591820310,397500.0,365000.0
7974200820,696000.0,865000.0
7701450110,925000.0,1038000.0
9522300010,1460000.0,1490000.0
9510861140,712000.0,711000.0


In [22]:
cross_val_score(regressor, X, y, cv=10)

array([0.80490686, 0.78966682, 0.74117872, 0.79768956, 0.66706778,
       0.65673401, 0.74576922, 0.75929571, 0.76071528, 0.81460161])

In [23]:
# Tree score
regressor.score(X, y, sample_weight=None)

0.919247840219902

In [24]:
# Machinbe learning model Random forest

In [25]:
from sklearn.ensemble import RandomForestRegressor

In [26]:
regressor_forest = RandomForestRegressor(max_depth=2, random_state=79)

In [33]:
y_pred = regressor_forest.fit(X_train, y_train).predict(X_test)
results_forest = pd.DataFrame({"Prediction": y_pred, "Actual": y_test})
results_forest.head(5)

,Prediction,Actual
id,,
2591820310,3.155011e+05,365000.0
7974200820,5.308580e+05,865000.0
7701450110,9.123269e+05,1038000.0
9522300010,1.421313e+06,1490000.0
9510861140,8.094779e+05,711000.0


In [28]:
cross_val_score(regressor_forest, X, y, cv=10)

array([0.51834514, 0.53612901, 0.55970558, 0.49169129, 0.51062762,
       0.54908181, 0.56218064, 0.55970309, 0.55793741, 0.51878254])

In [29]:
# Forest score
regressor_forest.score(X, y, sample_weight=None)

0.5365526911901541